In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from numba import jit

Dla zjawiska perkolacji węzłów potrzebujemy grafu (np. sieci regularnej), którego każdy z węzłów jest zajęty
z prawdopodobieństwem p bądź „pusty” z prawdopodobienstwem (1 − p). Zjawisko to przebadany na sieci kwadra-
towej z otwarymi (swobodnymi) warunkami brzegowymi.
Będziemy chcieli znaleźć prawdopodobieństwo perkolacji W (p; L) (wrapping probability) w zależności od liniowe-
go rozmiaru układu L i prawdopodobieństwa okupacji (wypełnienia, zajętości) pojedynczego węzła p. Prawdopod-
bieństwo perkolacji W określa prawdopodobieństwo, że przy wypełnienuu pL2 spośród L2 dostępnych w układzie
węzłów istnieje nieprzerwana ścieżka łącząca lewy brzeg z prawym albo górny z dolnym. Ograniczymy się do ba-
dania połączenia góra–dół (bo bardzo łatwo automatycznie wykrywać połączenie góra/dół). Dla małych układów
można to zrobić metodą brutalnej siły, dla dużych już nie, bo wypełnienie siatki kwadratowej o boku L = 100 przy
zapełnieniniu jej w połowie można zrobić na ( 104 / 5000) ≈ 1.6 × 103008 sposobów (tak, jedynka z ponad trzema tysiącami
zer).
Do odpowiedzi na powyższe pytanie potrzebujemy algorytmu identyfikacji zajętych węzłów należących do tego
samego klastra.
Posłużymy się (niezręcznie implementowanym) algorytmem Hoshena–Kopelmana. Algorytm pozwala zajętym
węzłom przypisać etykiety, w ten stposób, że węzły należące do tego samego klastra mają takie same etykiety a węzły
z różnych klastrów mają różne etykiety. Do wyznaczenia W (p; L) generujemy R przykładów sieci i sprawdzamy w ilu
(N (p; L)) z nich mamy etykiety węzłów z górnej krawędzi w dolnej. Wówczas W (p; L) = N (p; L)/R.
Alorytm Hoshena–Kopelmana każe przejść sekwencyjnie przez siatkę i za każdym razem, gdy napotykamy zajęty
węzeł (x, y) sprawdzamy czy ma on zajętego sąsiada w (x − 1, y) bądź (x, y − 1) (uroda algorytmu zasadza się
na tym, że pierwszą iterację etykietowania można robić równolegle z wypełnianiem sieci. Będziemy potrzebować
drugiej iteracji). Jeśli nie ma, to węzłowi przypisujemy pierwszą jeszcze niewykorzystaną etykietę. Jeśli ma, to węzeł
etykietujemy mniejszą spośród już przypisanych zajętym sąsiadom etykietom i (o ile jest to sytuacja, że obaj sąsiedzi
są zajęci) zapamiętujemy, że (konsekwentie, wstecz) etykietę wyższą z tych dwóch trzeba zastąpić niższą.

### Zadanie 1 (50 pkt.): 
Wygeneruj siatkę o boku L = 16 wypełnioną z p = 0,4; 0,6 oraz 0,8. Wypisz siatkę oraz ety-
kiety węzłów przypisanych algorytmem Hoshena–Kopelmana. Odpowiedz czy wygenerowana siatka perkoluje
czy nie.

In [90]:
L = 16
p = [0.4, 0.6, 0.8]
SIATKA = np.zeros((L+1, L+1), dtype=int)
ETYKIETY = np.zeros((L+1, L+1), dtype=int)


def losuj(prob):
    for i in range(L+1):
        for j in range(L+1):
            if random.random() < prob:
                SIATKA[i, j] = 1
            else:
                SIATKA[i, j] = 0

losuj(p[1])
SIATKA[0, :] = 0
SIATKA[:, 0] = 0
SIATKA

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1],
       [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1],
       [0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,

In [95]:
etykieta = 1
tab = []
for i in range(L+1):
    for j in range(L+1):
        if SIATKA[i, j] == 1:
            if SIATKA[i-1, j] == 0 and SIATKA[i, j-1] == 0:
                ETYKIETY[i, j] = etykieta
                etykieta += 1
            elif SIATKA[i-1, j] == 1 and SIATKA[i, j-1] == 1:
                if ETYKIETY[i-1, j] < ETYKIETY[i, j-1]:
                    ETYKIETY[i, j] = ETYKIETY[i-1, j]
                else:
                    ETYKIETY[i, j] = ETYKIETY[i, j-1]
            elif SIATKA[i-1, j] == 1 and SIATKA[i, j-1] == 0:
                ETYKIETY[i, j] = ETYKIETY[i-1, j]
            elif SIATKA[i-1, j] == 0 and SIATKA[i, j-1] == 1:
                ETYKIETY[i, j] = ETYKIETY[i, j-1]

for i in range(L, 1, -1):
    for j in range(L, 1, -1):
        if ETYKIETY[i-1, j] < ETYKIETY[i, j-1] and ETYKIETY[i-1, j] != 0 and ETYKIETY[i, j-1] != 0:
            ETYKIETY[i, j-1] = ETYKIETY[i-1, j]
        elif ETYKIETY[i-1, j] > ETYKIETY[i, j-1] and ETYKIETY[i-1, j] != 0 and ETYKIETY[i, j-1] != 0:
            ETYKIETY[i-1, j] = ETYKIETY[i, j-1]

print(ETYKIETY[1:,1:])

[[ 0  1  0  1  1  0  1  1  0  0  0  4  4  4  0  4]
 [ 1  1  1  1  0  1  1  0  1  8  4  0  4  4  4  4]
 [ 1  1  1  1  1  1  0  1  0  8  8  0  4  0  4  4]
 [ 1  1  1  1  1  0  1  9  0  8  8  4  4  0  0  4]
 [ 0  0  1  0  1  1  0  1  0  8  0  0  4  0  4  0]
 [ 0  0  0  1  0  1  1  0  8  8  0  4  0  0  0 15]
 [ 0 16 12  0  0  0  0 13  8  8  0 14 14 14  0 15]
 [ 0  0  0 18  0 18 17  0  8  0 14  0  0  0 15  0]
 [22 22  0 18 18  0 18 13  0  0 20  0 23 19 21 21]
 [22 22 18  0 18 18  0 19  0 20 19  0 19 21  0  0]
 [ 0  0 22 18 18  0 19 19  0 19 19 19  0  0 26  0]
 [27 22  0  0 18  0 19 19 19 19 19 19  0  0 19  0]
 [27 27 27  0  0 25 19  0 19 19 19 19 19 19 19 19]
 [ 0  0  0 29 28 25  0 19  0  0 19 19 19  0  0  0]
 [31 31  0  0 28  0  0  0 32 19  0  0 19  0 33 33]
 [ 0 31  0  0  0  0 34  0 32  0  0  0  0  0 33 33]]


In [96]:
counter = 0

for i in range(1, L+1):
    for j in range(1, L+1):
        if ETYKIETY[1][i] == ETYKIETY[-1][j] and ETYKIETY[1][i] != 0:
            counter += 1
counter

0

In [103]:
SIATKA = np.zeros((L+1, L+1), dtype=int)
ETYKIETY = np.zeros((L+1, L+1), dtype=int)
losuj(p[1])
SIATKA[0, :] = 0
SIATKA[:, 0] = 0
SIATKA

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
       [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,

In [104]:
etykieta = 1
tab = []

for i in range(1,L+1):
    for j in range(1,L+1):
        if SIATKA[i, j] == 1:
            if SIATKA[i-1, j] == 0 and SIATKA[i, j-1] == 0:
                ETYKIETY[i, j] = etykieta
                etykieta += 1
            elif SIATKA[i-1, j] == 1 and SIATKA[i, j-1] == 1:
                if ETYKIETY[i-1, j] < ETYKIETY[i, j-1]:
                    ETYKIETY[i, j] = ETYKIETY[i-1, j]
                    ETYKIETY[i, j-1] = ETYKIETY[i-1, j]
                else:
                    ETYKIETY[i, j] = ETYKIETY[i, j-1]
                    ETYKIETY[i-1, j] = ETYKIETY[i, j-1]
            elif SIATKA[i-1, j] == 1 and SIATKA[i, j-1] == 0:
                ETYKIETY[i, j] = ETYKIETY[i-1, j]
                
            elif SIATKA[i-1, j] == 0 and SIATKA[i, j-1] == 1:
                ETYKIETY[i, j] = ETYKIETY[i, j-1]

for i in range(L, 1, -1):
    for j in range(L, 1, -1):
        if ETYKIETY[i-1, j] < ETYKIETY[i, j-1] and ETYKIETY[i-1, j] != 0 and ETYKIETY[i, j-1] != 0:
            ETYKIETY[i, j-1] = ETYKIETY[i-1, j]
        elif ETYKIETY[i-1, j] > ETYKIETY[i, j-1] and ETYKIETY[i-1, j] != 0 and ETYKIETY[i, j-1] != 0:
            ETYKIETY[i-1, j] = ETYKIETY[i, j-1]
            
print(ETYKIETY[1:,1:])
# print(ETYKIETY)

[[ 1  0  2  2  2  0  3  3  0  0  0  0  4  0  4  0]
 [ 1  0  2  2  0  0  3  3  0  6  6  4  4  4  4  4]
 [ 1  0  0  0  7  0  3  3  0  6  4  0  0  4  0  0]
 [ 0  8  0  7  7  0  3  0  6  0  3  0  0  0  0 11]
 [ 8  8  7  7  0  3  3  0  0  3  3  3  0  3 11 11]
 [ 0  7  0  7  0  0  3  3  3  3  3  0  3  0 11  0]
 [ 0  8  0  0  0  3  3  0  3  3  3  3  0  0  3 11]
 [ 0  8  0  0  3  3  0  3  0  3  3  3  0  3 11  0]
 [ 8  8  0  3  3  3  3  3  3  3  3  3  3  0  3  0]
 [ 8  0  0  3  3  0  3  3  3  3  0  0  3  3  0  0]
 [ 0 23  3  3  0  3  0  3  3  3  0  3  0  0 26  3]
 [ 0  3  3  0  3  0  0  0  0  3  3  0  0  0  3 26]
 [ 3  3  3  3  0  3 29  3  3  3  0  3  0  3 26 26]
 [ 0  3  3  3  3  0  3  3  3  3  3  3  3  0 26 26]
 [ 3  0  3  3  3  3  3  0  0  3  0  0  0 26 26  0]
 [ 0  3  3  3  3  3  0 35  0  0 36  0 26 26 26 26]]


In [105]:
counter = 0

for i in range(0, L+1):
    for j in range(0, L+1):
        if ETYKIETY[1][i] == ETYKIETY[-1][j] and ETYKIETY[1][i] != 0:
            counter += 1
counter

10

In [106]:
SIATKA = np.zeros((L, L), dtype=int)
ETYKIETY = np.zeros((L, L), dtype=int)
losuj(p[2])
etykieta = 1

SIATKA

IndexError: index 16 is out of bounds for axis 1 with size 16

In [107]:

for i in range(L):
    for j in range(L):
        if SIATKA[i, j] == 1:
            if SIATKA[i-1, j] == 0 and SIATKA[i, j-1] == 0:
                ETYKIETY[i, j] = etykieta
                etykieta += 1
            elif SIATKA[i-1, j] == 1 and SIATKA[i, j-1] == 1:
                if ETYKIETY[i-1, j] < ETYKIETY[i, j-1]:
                    ETYKIETY[i, j] = ETYKIETY[i-1, j]
                    ETYKIETY[i, j-1] = ETYKIETY[i-1, j]
                else:
                    ETYKIETY[i, j] = ETYKIETY[i, j-1]
                    ETYKIETY[i-1, j] = ETYKIETY[i, j-1]
            elif SIATKA[i-1, j] == 1 and SIATKA[i, j-1] == 0:
                ETYKIETY[i, j] = ETYKIETY[i-1, j]
            elif SIATKA[i-1, j] == 0 and SIATKA[i, j-1] == 1:
                ETYKIETY[i, j] = ETYKIETY[i, j-1]

for i in range(L, 1, -1):
    for j in range(L, 1, -1):
        if ETYKIETY[i-1, j] < ETYKIETY[i, j-1] and ETYKIETY[i-1, j] != 0 and ETYKIETY[i, j-1] != 0:
            ETYKIETY[i, j-1] = ETYKIETY[i-1, j]
        elif ETYKIETY[i-1, j] > ETYKIETY[i, j-1] and ETYKIETY[i-1, j] != 0 and ETYKIETY[i, j-1] != 0:
            ETYKIETY[i-1, j] = ETYKIETY[i, j-1]
            
ETYKIETY

IndexError: index 16 is out of bounds for axis 1 with size 16

### Zadanie 2 (30 pkt.): 
Zautomatyzuj proces odpowiadania na pytanie czy siatka perkoluje czy nie (wyznaczania
N (p)). Dla L = 16 wygeneruj po R = 1000 siatek dla p od 0,4 do 0,8 co 0,01. Sporządź wykres W (p; L =
16) = N (p)/R.

### Zadanie 3 (20 pkt.): 
Powtórz obliczenia z zadania 2 dla L = 32. Nałóż je na wykres z poprzedniego zadania
(dwie krzywe na jednym rysunku: W (p; L = 16) oraz W (p; L = 32)).